# CHEQUEOS POST LIQUIDACION

### Obtencion de Datos

In [1226]:
import pyodbc as mssql
import numpy as np
import pandas as pd
%config IPCompleter.greedy=True
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from IPython.display import Markdown as md
import xlsxwriter as xw

In [1227]:
liquidacion = '2022-04'
minimo_general = 14102.10
minimo_policias = 44538.11
promedio_incrementos = 10.00/100.00
excluir_reparticiones_minimo_garantizado = [17, 26]

altas_desde = ''
codigos_bruto = ['0Q','0R','0N','0S','LC','OC','PC','QC','TC','U0','MC','PA','TE','XX','PX','MX','YX','NX','1X']


In [1228]:
# archivo de salida
writer = pd.ExcelWriter("Resumen-" + liquidacion + ".xlsx", "xlsxwriter")
wb = writer.book
formato_moneda = wb.add_format({'num_format': '$ #,##0.00'})
formato_porcentaje = wb.add_format({'num_format': '#,##0.00 %'})


In [1229]:
conn = mssql.connect("DRIVER={SQL SERVER};SERVER=IPSBD;DATABASE=LIQUIDACION;;Trusted_Connection=Yes;")

#### Obtención de la fecha de alta de la liquidacion seleccionada

In [1230]:
from datetime import datetime
altas_desde = pd.read_sql("""SELECT create_date FROM SYS.DATABASES
WHERE NAME like '%{liquidacion}%'""".format(liquidacion=liquidacion),conn)["create_date"][0].strftime('%d/%m/%Y')
print("ALTAS DESDE: {altas_desde}".format(altas_desde=altas_desde))

ALTAS DESDE: 19/04/2022


#### Obtención de Datos Generales

In [1231]:
codigos = pd.read_sql( "SELECT * FROM CODIGOS", conn, index_col="CODIGO")
reparticion = pd.read_sql( "SELECT * FROM REPARTICION", conn, index_col="Cod" )
tipos_beneficio = pd.read_sql ( "SELECT * FROM BENEFICIO", conn, index_col="Cod" )

### Auditorias desde la fecha de creacion de la liquidacion de referencia
auditorias = pd.read_sql( """SELECT CONTROL FROM AUDIT A WHERE A.ORIGEN='D' AND 
A.FECHA > CONVERT(int, CONVERT(datetime, '{altas_desde}'))+2""".format(altas_desde=altas_desde), conn, index_col="CONTROL")

beneficios = pd.read_sql( """SELECT CONTROL, PLLA, ORDEN, AFILIADO, BENEFICIO, COD_BENEFICIO, LETRA,
CASE WHEN COD_BENEFICIO in (7,25,27,29) then DBO.PORCENTAJE_COMPARTIDA(CONTROL) ELSE 1 END AS PORCENTAJE
FROM Liquidacion.dbo.DatosAfiliado
WHERE HABILITADO='S'""", conn, index_col="CONTROL")
beneficios[["COD_BENEFICIO"]]=beneficios[["COD_BENEFICIO"]].astype("int64")
### Excluímos las auditorias de altas y modificaciones para evitar las "deformaciones"
beneficios_sin_altas = beneficios[~beneficios.index.isin(auditorias.index)]
liquidacion_ben = pd.read_sql(""" SELECT * FROM LIQUIDACION.DBO.LIQUIDACION""", conn, index_col=["PLLA", "ORDEN", "AFILIADO"])
beneficios_liq = pd.read_sql("SELECT * FROM [{liquidacion}].DBO.DATOSAFILIADO".format(liquidacion=liquidacion), \
                             conn, index_col=["CONTROL"])
liquidacion_liq = pd.read_sql("SELECT * FROM [{liquidacion}].DBO.LIQUIDACION".format(liquidacion=liquidacion), \
                                conn, index_col=["PLLA", "ORDEN", "AFILIADO"])



### Determinacion de brutos

#### Determinacion de los brutos actuales

In [1232]:
### determinacion de la tabla de brutos de liquidacion activa
brutos_act = beneficios_sin_altas.reset_index().merge(liquidacion_ben, left_on=["PLLA", "ORDEN", "AFILIADO"],\
                              right_on=["PLLA", "ORDEN", "AFILIADO"])
brutos_act = brutos_act[brutos_act["CODIGO"].isin(codigos_bruto)].groupby(["CONTROL"]).sum("MONTO")[["MONTO"]]
# agrego aquellos que no tienen codigos de bruto
brutos_act = brutos_act.join(beneficios_sin_altas,  how="outer")
brutos_act["MONTO"].fillna(value=0.0, inplace=True)
brutos_act["BRUTO"] = brutos_act["MONTO"] + brutos_act["BENEFICIO"]
# dejo solo la columna del bruto recien calculada
brutos_act=brutos_act[["BRUTO","BENEFICIO", "COD_BENEFICIO", "LETRA"]]


#### Determinación de los brutos de referencia

In [1233]:
###determinacion de la tabla de brutos de la liquidacion de referencia 
brutos_liq = beneficios_liq.join(liquidacion_liq, on=["PLLA","ORDEN","AFILIADO"],how="inner", rsuffix=".LIQ")\
        .join(codigos, on=["CODIGO"], how="inner", rsuffix=".COD")
brutos_liq = brutos_liq[brutos_liq["CODIGO"].isin(codigos_bruto)].groupby(by="CONTROL").sum()[["MONTO"]]
brutos_liq = brutos_liq.join(beneficios_liq, on="CONTROL", how="outer")
brutos_liq["MONTO"].fillna(value=0.0, inplace=True)
brutos_liq["BRUTO"] = brutos_liq["MONTO"] + brutos_liq["BENEFICIO"]
brutos_liq=brutos_liq[["CONTROL", "BRUTO", "BENEFICIO"]]
brutos_liq.set_index(keys="CONTROL", inplace=True)
brutos_liq=brutos_liq.sort_values(by="CONTROL")


### Comparativa de Brutos y Beneficios

In [1234]:
###  comparativa 

brutos = brutos_liq.join( brutos_act, lsuffix="_liq", rsuffix="_act" )
brutos["DIF_BRUTOS"] = (brutos["BRUTO_act"]-brutos["BRUTO_liq"])/brutos["BRUTO_liq"]
brutos["DIF_BENEFICIOS"] = (brutos["BENEFICIO_act"]-brutos["BENEFICIO_liq"])/brutos["BENEFICIO_liq"]
diferencia_brutos=brutos.loc[(brutos["DIF_BRUTOS"] > promedio_incrementos) | \
                             (brutos["DIF_BENEFICIOS"] > promedio_incrementos), ["LETRA", "COD_BENEFICIO", "DIF_BRUTOS","DIF_BENEFICIOS", ]] \
                            .sort_values(by="DIF_BRUTOS", ascending=False)
diferencia_brutos=diferencia_brutos.merge(reparticion, left_on="LETRA", right_on="Cod", how="inner")
diferencia_brutos=diferencia_brutos.join(tipos_beneficio, on="COD_BENEFICIO", rsuffix=".tb")
diferencia_brutos=diferencia_brutos[["Detalle", "Detalle.tb", "DIF_BRUTOS", "DIF_BENEFICIOS" ]]
diferencia_brutos=diferencia_brutos.rename(columns={"Detalle":"Reparticion", "Detalle.tb":"Tipo_Beneficio"})                  

diferencia_brutos.to_excel(writer, "Diferencia Brutos",startrow=0, header=True, index=True)

print(diferencia_brutos)

(max_row, max_col)= diferencia_brutos.shape
column_settings = []
column_settings = np.append(column_settings, {'header': "CONTROL"})
column_settings = np.append(column_settings, [{'header': column} for column in inf_rep_letra.columns])

worksheet = writer.sheets["Diferencia Brutos"]
worksheet.add_table(0, 0, max_row+1, max_col, {'columns': column_settings, 'autofilter': False, \
                                             'banded_rows': False, "style": None})

worksheet.set_column("A:A", 35)
worksheet.set_column("D:E", 15, formato_porcentaje )




                       Reparticion                Tipo_Beneficio  DIF_BRUTOS  \
0                          Docente  Jubilación Ordinaria Docente        1.14   
1                          Docente  Jubilación Ordinaria Docente        0.83   
2                          Docente  Jubilación Ordinaria Docente        0.32   
3                          Docente  Jubilación Ordinaria Docente        0.24   
4                          Docente  Jubilación Ordinaria Docente        0.24   
...                            ...                           ...         ...   
17276  Minist. Educ Ens Med y Priv      Jubilación por Invalidez        0.08   
17277  Minist. Educ Ens Med y Priv          Jubilación Ordinaria        0.08   
17278  Minist. Educ Ens Med y Priv          Jubilación Ordinaria        0.08   
17279  Minist. Educ Ens Med y Priv              Pension Derivada        0.08   
17280  Minist. Educ Ens Med y Priv          Jubilación Ordinaria        0.08   

       DIF_BENEFICIOS  
0              

0

#### Diferencia por reparticion

In [1235]:
### POR REPARTICION ###

inf_rep_letra = brutos.merge(reparticion, left_on="LETRA", right_on="Cod", how="inner")
inf_rep_letra = inf_rep_letra[['Detalle', 'BRUTO_liq', 'BENEFICIO_liq', 'BRUTO_act', 'BENEFICIO_act']].groupby(by="Detalle").sum()
inf_rep_letra["DIF_BRUTOS"] = (inf_rep_letra["BRUTO_act"]-inf_rep_letra["BRUTO_liq"])/inf_rep_letra["BRUTO_liq"]
inf_rep_letra["DIF_BENEFICIOS"] = (inf_rep_letra["BENEFICIO_act"] - inf_rep_letra["BENEFICIO_liq"])/inf_rep_letra["BENEFICIO_liq"]
inf_rep_letra=inf_rep_letra[(inf_rep_letra["DIF_BRUTOS"]!=0) | (inf_rep_letra["DIF_BENEFICIOS"]!=0)]

inf_rep_letra.to_excel(writer, "Dif Brutos Por Rep")

(max_row, max_col)= inf_rep_letra.shape

column_settings = []
column_settings = np.append(column_settings, {'header': "Reparticion"})
column_settings = np.append(column_settings, [{'header': column} for column in  inf_rep_letra.columns])

worksheet = writer.sheets["Dif Brutos Por Rep"]
worksheet.add_table(0, 0, max_row, max_col, {'columns': column_settings, 'autofilter': False, \
                                             'banded_rows': False, "style": None})

worksheet.set_column("A:A", 35)
worksheet.set_column("F:G", 15, formato_porcentaje )
worksheet.set_column("B:E", 15, formato_moneda)



0

### Mínimos

#### Mínimos Administración Central y Organismos

In [1236]:
# minimos = beneficios.join( beneficios_liq, on="CONTROL", lsuffix=".BEN", rsuffix=".BENLIQ" )\
#             .join(reparticion, on="LETRA" , rsuffix=".REP", )\
#             .join(brutos, on="CONTROL", rsuffix=".BRT")

# minimos[minimos["BRUTO_act"] < minimo_general]
# minimos["ZX_act"] = np.where((~minimos["LETRA"].isin([17,26]) & (minimos["BRUTO_act"] < minimo_general*minimos["PORCENTAJE"])), minimo_general*minimos["PORCENTAJE"]-minimos["BRUTO_act"], 0.0)
# minimos.fillna(0, inplace=True)

# minimos_pagados = beneficios_liq.join(liquidacion_liq, on=["PLLA","ORDEN","AFILIADO"],how="inner", rsuffix=".LIQ")\
#         .join(codigos, on=["CODIGO"], how="inner", rsuffix=".COD")

# minimos_pagados=minimos_pagados.loc[minimos_pagados["CODIGO"].isin(['ZX']), ["MONTO"]].groupby("CONTROL").sum("MONTO").rename(columns={"MONTO":"ZX_liq"})

# minimos=minimos.join(minimos_pagados, on="CONTROL", rsuffix=".MINPAG")
# minimos.fillna(0.0, inplace=True)
# minimos=minimos.loc[(np.abs(minimos["ZX_act"]-minimos["ZX_liq"])>1)  ,["BRUTO_act", "ZX_act", "BRUTO_liq", "ZX_liq"]]

# minimos.to_excel(writer, "Minimos_Adm_Central")



#### Mínimos Retiros Policiales y Carcelarios

In [1237]:
# minimos = beneficios.join( beneficios_liq, on="CONTROL", lsuffix=".BEN", rsuffix=".BENLIQ" )\
#             .join(reparticion, on="LETRA" , rsuffix=".REP", )\
#             .join(brutos, on="CONTROL", rsuffix=".BRT")\
#             .join(tipos_beneficio, on="COD_BENEFICIO", rsuffix=".tb")
            

# minimos[minimos["BRUTO_act"] < minimo_policias]
# minimos["PX_act"] = np.where((minimos["LETRA"].isin([17,26]) & (minimos["BRUTO_act"] < minimo_policias*minimos["PORCENTAJE"])), minimo_policias*minimos["PORCENTAJE"]-minimos["BRUTO_act"], 0.0)
# minimos.fillna(0, inplace=True)

# minimos_pagados = beneficios_liq.join(liquidacion_liq, on=["PLLA","ORDEN","AFILIADO"],how="inner", rsuffix=".LIQ")\
#         .join(codigos, on=["CODIGO"], how="inner", rsuffix=".COD")

# minimos_pagados=minimos_pagados.loc[minimos_pagados["CODIGO"].isin(['PX']), ["MONTO"]]\
#         .groupby("CONTROL").sum("MONTO")

# minimos=minimos.join(minimos_pagados, on="CONTROL", rsuffix=".MINPAG").rename(columns={"MONTO":"PX_liq", "Detalle":"Reparticion", "Detalle.tb":"TIPO_BEN"})
# minimos.fillna(0.0, inplace=True)

# minimos=minimos.loc[(np.abs(minimos["PX_act"]-minimos["PX_liq"])>1)  ,["Reparticion", "TIPO_BEN", "BRUTO_act", "PX_act", "BRUTO_liq", "PX_liq"]]

# #minimos.to_excel(writer, "Minimos Retiros Policiales")



## Brutos Altas y Modificaciones


## Cerrar archivo excel y guardar

In [1238]:
### Guardado y cierre del archivo excel
writer.save()